<center><h1>Laboratory 4 - MongoDB</h1><h5>Jan Gołda, 291431</h5></center>
<br>

# MongoDB Shell version
---
### Setup
```
> use jeopardy
```

### Query 1
Quesions from rounds 'double' and 'final' in show number 2047, ordered by question value.  
(ordered is lexicographical order, since given data is broken and sometimes uses "," ect.)  
``` js
db.question.find({
    round: {\$in: ["Double Jeopardy!", "Final Jeopardy!"]},
    show_number: "2047"
  }, {
    value: 1,
    question: 1,
    answer: 1,
    _id: 0
}).sort({value: 1});
```  

Result example (31 objcts in total):
```
{ "question" : "'The name of this river, famous in song, may be a corruption of the Spanish for \"little Saint John\"'", "value" : null, "answer" : "the Swanee" }
...
```


### Query 2
List of questions values with their occurrence count, ordered (descending) by count.  
``` js
db.question.aggregate([
  { \$match: {show_number: "2047"} },
  { \$group: {_id: "$value", count: {$sum: 1}} },
  { \$sort: {count: -1} }
]);
```  

Results:
``` js
{ "_id": "\$200", "count": 12 }
{ "_id": "\$400", "count": 12 }
{ "_id": "\$300", "count": 6 }
{ "_id": "\$100", "count": 6 }
{ "_id": "\$800", "count": 6 }
{ "_id": "\$500", "count": 5 }
{ "_id": "\$1000", "count": 5 }
{ "_id": "\$600", "count": 5 }
{ "_id": null, "count": 1 }
{ "_id": "\$2,500", "count": 1 }
{ "_id": "\$1,500", "count": 1 }
{ "_id": "\$3,000", "count": 1 }
```

### Query 3
Counts of empty, one-word and many-words answers in Jeopardy history.  
``` js
db.question.mapReduce(
  function() {
    let words = this.answer.split(" ").length;
    
    if(words == 0)
      emit("empty", 1);
    if(words == 1)
      emit("one_word", 1);
    if(words > 1)
      emit("many_words", 1);
  },
  function(key, values) { return Array.sum(values) },
  {out: "ans_query_3"}
);
db.ans_query_3.find();
```  

Results:
``` js
{
	"result" : "ans_query_3",
	"timeMillis" : 2049,
	"counts" : {
		"input" : 216930,
		"emit" : 216930,
		"reduce" : 4340,
		"output" : 2
	},
	"ok" : 1
}
```
``` js
{ "_id" : "many_words", "value" : 124229 }
{ "_id" : "one_word", "value" : 92701 }
```

# Python3 version
---
### Setup

In [1]:
from pymongo import MongoClient
from bson.code import Code

client = MongoClient()
question = client.jeopardy.question

### Query 1
Quesions from rounds 'double' and 'final' in show number 2047, ordered by question value.  
(ordered is lexicographical order, since given data is broken and sometimes uses "," ect.) 

In [2]:
result = question.find({
    "round": {"$in": ["Double Jeopardy!", "Final Jeopardy!"]},
    "show_number": "2047"
  }, {
    "value": 1,
    "question": 1,
    "answer": 1,
    "_id": 0
  }).sort("value")

for q in result:
  print("{value}\t{question:.40}...\t{answer}".format(**q))

None	'The name of this river, famous in song,...	the Swanee
$1000	'This nucleic acid occurs in 3 forms: me...	RNA
$1000	'First performed in 1905, this very shor...	The Dying Swan
$1000	'About 3000 B.S. the Sumerians invented ...	cuneiform
$1000	'This somewhat coarse root vegetable is ...	a rutabaga
$1000	'This chief once called "The Apache Napo...	Cochise
$2,500	'He followed his first novel, "Appointme...	John O\'Hara
$200	'In 1991 B.C. Amenemhet, a former vizier...	Egypt
$200	'"Homage to the Queen", a tribute to her...	Elizabeth II
$200	'The summer varieties of this gourd-like...	squash
$200	'Unlike most birds, ratites like the ost...	fly
$200	'This novelist's youthful voyages provid...	Conrad
$200	'In 1777 Chief Joseph Brant led his fell...	the Revolutionary War
$3,000	'In 1919 this national assembly met in t...	Weimar
$400	'Because his proposals for constitutiona...	Charles de Gaulle
$400	'This dancer choreographed a new version...	Baryshnikov
$400	'A woman claiming to be this Lewis

### Query 2
List of questions values with their occurrence count, ordered (descending) by count.  

In [3]:
result = question.aggregate([
    { "$match": {"show_number": "2047"} },
    { "$group": {"_id": "$value", "count": {"$sum": 1}} },
    { "$sort": {"count": -1} }
  ])

for q in result:
  print("{_id}\tx{count}".format(**q))

$200	x12
$400	x12
$300	x6
$100	x6
$800	x6
$500	x5
$1000	x5
$600	x5
None	x1
$2,500	x1
$1,500	x1
$3,000	x1


### Query 3
Counts of empty, one-word and many-words answers in Jeopardy history.  

In [4]:
query_map = Code("""
  function() {
    let words = this.answer.split(" ").length;
    
    if(words == 0)
      emit("empty", 1);
    if(words == 1)
      emit("one_word", 1);
    if(words > 1)
      emit("many_words", 1);
  }
""")

In [5]:
query_reduce = Code("function(key, values) { return Array.sum(values) }")

In [6]:
result = question.map_reduce(query_map, query_reduce, "ans_query_3")

for q in result.find():
  print("{_id}\tx{value:.0f}".format(**q))

many_words	x124229
one_word	x92701


### Cleanup

In [7]:
client.close()

<span style="float: right">Jan Gołda, 291431</span>